# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится.

Необходимо построить модель с максимально большим значением *accuracy*. 

## Откройте и изучите файл

### Для начала импортируем все необходимые библиотеки

In [46]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.dummy import DummyClassifier

### Изучим датасет

In [23]:
data = pd.read_csv('/datasets/users_behavior.csv')

In [24]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [26]:
data['is_ultra'].sum()

985

Все данные заполнены, при этом их не столь много, 3214 строк, из них 985 клиентов используют тариф "Ультра".

## Разбейте данные на выборки

### Для начала разделим датасет на 3 части

Поскольку у нас есть лишь одна выборка, её необходимо разделить. Делить предлагаю на 3 части, обучающая, валидационная и тестовая, в соотношении 3:1:1

In [27]:
data_train, data_valid = train_test_split(data, test_size=0.20, random_state=12345)

In [28]:
data_train, data_test = train_test_split(data_train, test_size=0.25, random_state=12345)

In [29]:
data_train.shape

(1928, 5)

In [30]:
data_valid.shape

(643, 5)

In [31]:
data_test.shape

(643, 5)

Данные разделении на соответствующие части. К содалению, не нашёл в библиотеках способа разбить данные сразу на три части, поэтому сначала датасет был разделён на 2 в соотношении 4 к 1, после чего больший был разделён в соотношении уже 3 к одному, что бы сохранить общенее соотношение 60%:20%:20% 

### Теперь разделим эти выборки

In [32]:
features_train = data_train.drop('is_ultra', axis=1)
target_train = data_train['is_ultra']
features_valid = data_valid.drop('is_ultra', axis=1)
target_valid = data_valid['is_ultra']
features_test = data_test.drop('is_ultra', axis=1)
target_test = data_test['is_ultra']

Все данные разделены, можно переходить к исследованиям.

In [33]:
features_train

,calls,minutes,messages,mb_used
2656,30.0,185.07,34.0,17166.53
823,42.0,290.69,77.0,21507.03
2566,41.0,289.83,15.0,22151.73
1451,45.0,333.49,50.0,17275.47
2953,43.0,300.39,69.0,17277.83
...,...,...,...,...
1043,106.0,796.79,23.0,42250.70
2132,18.0,117.80,0.0,10006.79
1642,87.0,583.02,1.0,11213.97
1495,63.0,408.68,63.0,24970.26


## Исследуйте модели

Поскольку у нас есть всего 2 тарифа на выбор, будем использовать методы классификации.

### Для начала предлагаю проверить качество модели с деревом решений с разным количеством глубины.

In [44]:
best_model = None
best_result = 0
for depth in range(1, 51):
    model = DecisionTreeClassifier(random_state=3214, max_depth=depth) 
    model.fit(features_train, target_train) 
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model = model
        best_result = result
print(best_result)
print(best_model)

0.7900466562986003
DecisionTreeClassifier(max_depth=5, random_state=3214)


С деревом решений мы получили наилучший результат 79% с глубиной дерева 5. В целом это уже соответствует условию задания, однако попробуем найти что-то лучше.

In [45]:
best_model = None
best_result = 0
for depth in range(1, 16):
    model = DecisionTreeClassifier(random_state=3214, max_depth=depth) 
    model.fit(features_train, target_train) 
    print('Обучающая выборка', model.score(features_train, target_train))
    print('Валидационная выборка', model.score(features_valid, target_valid))

Обучающая выборка 0.758298755186722
Валидационная выборка 0.7480559875583204
Обучающая выборка 0.79201244813278
Валидационная выборка 0.7838258164852255
Обучающая выборка 0.8117219917012448
Валидационная выборка 0.7869362363919129
Обучающая выборка 0.8205394190871369
Валидационная выборка 0.7869362363919129
Обучающая выборка 0.8278008298755186
Валидационная выборка 0.7900466562986003
Обучающая выборка 0.8335062240663901
Валидационная выборка 0.7776049766718507
Обучающая выборка 0.8506224066390041
Валидационная выборка 0.7838258164852255
Обучающая выборка 0.8661825726141079
Валидационная выборка 0.7807153965785381
Обучающая выборка 0.875
Валидационная выборка 0.7807153965785381
Обучающая выборка 0.8910788381742739
Валидационная выборка 0.7713841368584758
Обучающая выборка 0.9024896265560166
Валидационная выборка 0.7636080870917574
Обучающая выборка 0.9149377593360996
Валидационная выборка 0.7682737169517885
Обучающая выборка 0.9242738589211619
Валидационная выборка 0.749611197511664
Обу

Так очень хорошо становиться заметно переобучение модели, заметно, что после значения максимальной глубины - 5 значение на валидационной выборке начинает резко снижаться.

### Теперь проверим случайный лес на качество с разным количеством деревьев и глубиной.

In [35]:
best_model = None
best_result = 0
for est in range(1, 21):
    for depth in range(1, 21):
        for sempl in range(1, 6):
            model = RandomForestClassifier(random_state=3214, n_estimators=est, max_depth=depth, min_samples_leaf=sempl)
            model.fit(features_train, target_train)
            result = model.score(features_valid, target_valid)
            if result > best_result:
                best_model = model
                best_result = result
print(best_result)
print(best_model)        

0.7480559875583204
0.7480559875583204
0.7480559875583204
0.7480559875583204
0.7480559875583204
0.7822706065318819
0.7822706065318819
0.7822706065318819
0.7822706065318819
0.7822706065318819
0.7838258164852255
0.7838258164852255
0.7838258164852255
0.7838258164852255
0.7838258164852255
0.7713841368584758
0.7698289269051322
0.7698289269051322
0.7698289269051322
0.7698289269051322
0.7853810264385692
0.7838258164852255
0.7884914463452566
0.7822706065318819
0.7853810264385692
0.7884914463452566
0.7807153965785381
0.7822706065318819
0.7776049766718507
0.7838258164852255
0.7807153965785381
0.7822706065318819
0.7838258164852255
0.7682737169517885
0.7916018662519441
0.776049766718507
0.7776049766718507
0.7667185069984448
0.7558320373250389
0.7713841368584758
0.7713841368584758
0.7667185069984448
0.7667185069984448
0.7651632970451011
0.7822706065318819
0.7682737169517885
0.7682737169517885
0.7558320373250389
0.776049766718507
0.7729393468118196
0.7698289269051322
0.7433903576982893
0.752721617418

Результат стал чуть лучше, с максимальной глубиной 9 и количеством деревьев 18 почти 81%,времени конечно занимает чуть больше, но не критично. Критично будет поставить большее кол-во оценщиков, расчёты начинают занимать слишком много времени. 

### Попробуем AdaBoost класификатор, возможно здесь он сможет помочь лучше.

In [37]:
best_model = None
best_result = 0
for est in range(1, 51):
    model = AdaBoostClassifier(random_state=3214, n_estimators=est) 
    model.fit(features_train, target_train) 
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model = model
        best_result = result
print(best_result)
print(best_model)

0.7962674961119751
AdaBoostClassifier(n_estimators=17, random_state=3214)


Лучший результат не достигнут, будем эксперементировать дальше.

### Далее попробуем Extra Tree, всё таки в названии есть экстра) кол-во деревьев здесь предлагаю оставить стандртным, их и так 100, попробуем менять только глубину.

In [38]:
best_model = None
best_result = 0
for depth in range(1, 51):
    model = ExtraTreeClassifier(random_state=3214, max_depth=depth) 
    model.fit(features_train, target_train) 
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model = model
        best_result = result
print(best_result)
print(best_model)

0.7993779160186625
ExtraTreeClassifier(max_depth=10, random_state=3214)


Лучше многих, однако, всё же недостаточно.

### Попробуем градиентный классификатор.

In [39]:
best_model = None
best_result = 0
for est in range(1, 21):
    for depth in range(1, 11):
        model = GradientBoostingClassifier(random_state=3214, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        result = model.score(features_valid, target_valid)
        if result > best_result:
            best_model = model
            best_result = result
print(best_result)
print(best_model)   

0.80248833592535
GradientBoostingClassifier(max_depth=5, n_estimators=20, random_state=3214)


Достаточно долго, да и результат не самый лучший. Ищем дальше.

### Попробуем линейный классификатор

In [40]:
best_model = None
best_result = 0
for it in range(1, 11):
    model = RidgeClassifier(random_state=3214, max_iter=it) 
    model.fit(features_train, target_train) 
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model = model
        best_result = result
print(best_result)
print(best_model)

0.7558320373250389
RidgeClassifier(max_iter=1, random_state=3214)


К сожалению, худший результат из всех(

### Вывод

Наилучшим образом себя показала модеь случайного леса с глубиной 9 и 18 оценщиками, почти 81% правильных ответов. Вероятно можно подобрать что-то лучше или настроить гиперпараметры случайного леса с большим диапазоном, однако расчёты при таких параметрах уже занимают остаточно большое количество времени, поэтому предлагаю остановиться именно на этой моделе.

## Проверим модель на тестовой выборке

Теперь проверим эту модель на тестовой выборке.

In [41]:
best_model = RandomForestClassifier(random_state=3214, n_estimators=18, max_depth=9)
best_model.fit(features_train, target_train)
best_model.score(features_test, target_test)

0.7807153965785381

В целом неплохо, 78% правильно предсказанных тарифов.

## Проверим модели на адекватность

К сожалению, не особо понимаю как правильно это нужно сделать. Сначала посчитал кол-во ошибок, но они ничего не дадут, учитывая то, что мы знаем процент правильных ответов. Вероятно здесь можно дать какой-то определённый набор данных для расчёта, только мне кажется, что писать его вручную это не совсем правильно, данных получиться мало и врядли получиться адекватно что-либо оценить. Возможно этот процесс можно как то автоматизировать или проверка подразумевает собой нечто другое. Поиски в интернете так же мало к чему привели.

In [48]:
dummy_model = DummyClassifier(random_state=3214)
dummy_model.fit(features_train, target_train)
dummy_model.score(features_test, target_test)

0.6889580093312597

Наивный алгоритм в данном случае даёт почти 69% точности, наша модель получилась точнее, а значит вполне адекватная)

## Общий вывод

По итогу, для предсказания тарифов лучше всего подошёл случайный лес, который дал 78% правильных ответов на тренировочных данных. Вероятно, ещё меняя параметры, можно достичь лучшего результата, однако это будет занимать достаточно большое количество времени. Так же вряд-ли получиться достичь 100% правильных ответов, ведь в выгрузке, да и в жизни, тарифы уже выбраны людьми и некоторые могут быть подобраны не по потребностям. Однако и 78% уже достаточно непохой результат для определения тарифа у клиента. Вероятно, в данном случае для классификации случайный лес был наиболее подходящим решением, остальные, даже при большем дипазоне гиперпараметров давали результат хуже.